<a href="https://colab.research.google.com/github/konstantinos-al/ml_project_quebec/blob/main/Preprocessing_(without_image_generator).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import glob
import xmltodict
from tensorflow.keras.utils import to_categorical
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

xmls_dir = os.path.join("/content/drive/MyDrive/Untitled_folder/anot")

imgs_dir = r"/content/drive/MyDrive/Untitled_folder/ima"

heads = os.makedirs('heads', exist_ok=True)

In [ ]:
# def visualize(**images):
#     """PLot images in one row."""
#     n = len(images)
#     plt.figure(figsize=(16, 5))
#     for i, (name, image) in enumerate(images.items()):
#         plt.subplot(1, n, i + 1)
#         plt.xticks([])
#         plt.yticks([])
#         plt.title(' '.join(name.split('_')).title())
#         plt.imshow(image)
#     plt.show()

In [ ]:
# Δημιουργούμε τα labels και κρατάμε κεφάλια


imgs_labels = []

for dir_path in glob.glob(xmls_dir):
    for xml_path in glob.glob(os.path.join(dir_path, '*.xml')):
        with open (xml_path, 'rb') as my_xml:
            xmld = xmltodict.parse(my_xml)
            tags = xmld['annotation']

            image = cv2.cvtColor(cv2.imread(os.path.join(imgs_dir, tags['filename']), 1), cv2.COLOR_BGR2RGB)
            mask = np.zeros(image.shape, dtype = 'int32')

            if 'object' in tags:

                if isinstance(tags['object'], list):
                    for i in tags['object']:
                        xmin = int(i['bndbox']['xmin'])
                        ymin = int(i['bndbox']['ymin'])
                        xmax = int(i['bndbox']['xmax'])
                        ymax = int(i['bndbox']['ymax'])
                        if i['name'] == 'With Helmet':
                            mask = cv2.rectangle(mask, (xmin,ymin), (xmax,ymax), (0,255,0), -1)
                            result = cv2.addWeighted(image, 1, mask, 0.9, 0.7, dtype = cv2.CV_8UC3)
                            imgs_labels.append([0, image[ymin:ymax, xmin:xmax]])
                            #visualize(image=image, result = result, head = image[ymin:ymax, xmin:xmax])
                           
                        else:
                            cv2.rectangle(mask, (xmin,ymin), (xmax,ymax), (255,0,0), -1)
                            result = cv2.addWeighted(image, 1, mask, 0.9, 0.7, dtype = cv2.CV_8UC3)
                            imgs_labels.append([1, image[ymin:ymax, xmin:xmax]])
                          
                            #plt.imsave(os.path.join('results', tags['filename']), result)
                    #visualize(image=image, result = result, head = image[ymin:ymax, xmin:xmax])
                if isinstance(tags['object'], dict):
                    xmin = int(tags['object']['bndbox']['xmin'])
                    ymin = int(tags['object']['bndbox']['ymin'])
                    xmax = int(tags['object']['bndbox']['xmax'])
                    ymax = int(tags['object']['bndbox']['ymax'])
                    if tags['object']['name'] == 'With Helmet':
                        cv2.rectangle(mask, (xmin,ymin), (xmax,ymax), (0,255,0), -1)
                        result = cv2.addWeighted(image, 1, mask, 0.9, 0.7, dtype = cv2.CV_8UC3)
                        imgs_labels.append([0, image[ymin:ymax, xmin:xmax]])
                        #visualize(image=image, result = result, head = image[ymin:ymax, xmin:xmax])
                    else:
                        cv2.rectangle(mask, (xmin,ymin), (xmax,ymax), (255,0,0), -1)
                        result = cv2.addWeighted(image, 1, mask, 0.9, 0.7, dtype = cv2.CV_8UC3)
                        imgs_labels.append([1, image[ymin:ymax, xmin:xmax]])
                    # plt.imsave(os.path.join('results', tags['filename']), result)
                    #visualize(image=image, result = result, head = image[ymin:ymax, xmin:xmax])
            else:
                pass



In [ ]:
# Υπάρχουν κάποιες φωτογρασφίες με λάθος τα annotations και για να μην σκάει προς το παρόν

def checking_for_zero_shapes(list):
    for i, label_image in enumerate(list):
        image = np.array(label_image[1]).astype(np.int32)
        if image.shape[0]==0 or image.shape[1]==0:
            list.pop(i)

    return list

In [ ]:
# Κάνουμε visualize μία φωτό + reshape, flatten και 0-1

imgs_labels = checking_for_zero_shapes(imgs_labels)
#imgs_labels.pop(1342) #H prohgoumenh sunarthsh de mporei na bgalei authn thn eikona, de kserw giati
imgs_labels = np.array(imgs_labels, dtype='object').reshape(-1,2,1)
#imgs_labels = np.array(imgs_labels).reshape(-1,2,1)
labels = imgs_labels[:,0].reshape(-1,1).astype(np.int32)
data = imgs_labels[:,1].flatten()
imgs = data/255
imgs = np.array(imgs)
#print(imgs)

visualize(MALAKAS_AL = imgs[0])

In [ ]:
# Εδώ απλά δημιουργούμε ένα csv με τα labels για να τσεκάρουμε μια γρήγορα τι παίζει με τις φωτό που φτιάξαμε (κομμένα κεφάλια)

with open('imgs_labels.csv', 'w') as file:
    for i, image_label in enumerate(zip(images, labels)):
        plt.imsave(os.path.join('heads', str(i)+'.png'), image_label[0])
        file.write('Label: {0} | Image: {1}.png\n'.format(image_label[1],i))
       
    file.close()